In [1]:
%run setup.py

# Condense the model grid into numpy binary objects

## Inverse-cubic, all sample solution.

In [5]:
grids = pd.read_csv('../hpc/coarse_v2/template/coarse_grid_input_params_v2.txt') 

In [6]:
from tqdm import tqdm
import astropy.table
from grid import get_model_Dnu
import scipy.interpolate

for i in range(0, 9): #

    start = i*1000
    end = (i+1)*1000-1 if (i+1)*1000-1 < (2**13) else 2**13
    sl = slice(start, end, 1)
    print(sl)
    
    
    def multi(inputs):
        it, track = inputs
        filepath = '../hpc/coarse_v2/pre_rgb_tip/complete_grid/index{:06.0f}.history.h5'.format(track['index'])
        if not os.path.exists(filepath): 
            return None

        cols = ['index', 'profile_number', 'flag_seismo', 
                'Yinit', 'Xinit', 'Zinit', 'fov_core', 'fov_shell', 'amlt', 
                'star_age', 'star_mass', 'luminosity', 'radius', 'Teff', 
                'log_g', 'FeH', 'delta_nu_scaling', 'numax_scaling', 'acoustic_cutoff']

        f = h5py.File(filepath, 'r')
        # print(filepath)
        cols = ['index', 'profile_number', 'flag_seismo', 
                'star_age', 'star_mass', 'luminosity', 'radius', 'Teff', 'delta_nu_scaling', 'numax_scaling', 'acoustic_cutoff', 'delta_Pg', 'log_g',
                'Yinit', 'Xinit', 'Zinit', 'center_h1', 'fov_core', 'fov_shell', 'amlt', 'FeH', 'log_LHe', 'log_Lnuc']


        atrack = Table([f[col][:] for col in cols], names=cols)
        idx = (atrack['flag_seismo']==1) & (atrack['numax_scaling']>15) & ((atrack['log_LHe']-atrack['log_Lnuc'])<-3)

        atrack = atrack[idx]
        atrack['density'] = atrack['star_mass']**0.5/atrack['radius']**-1.5
        atrack['numax'] = atrack['numax_scaling']
        atrack['Dnu'] = atrack['delta_nu_scaling']

        atrack['[M/H]'] = atrack['FeH']
        atrack['mode_l'] = np.zeros(len(atrack), dtype=object)
        atrack['mode_n'] = np.zeros(len(atrack), dtype=object)
        atrack['mode_freq_o'] = np.zeros(len(atrack), dtype=object)
        atrack['mode_freq'] = np.zeros(len(atrack), dtype=object)
        atrack['mode_inertia'] = np.zeros(len(atrack), dtype=object)
        
        # # # replace the acoustic_cutoff from mesa with numax_scaling
        atrack['acoustic_cutoff'] = atrack['numax_scaling']
        atrack['Dnu_freq_o'] = np.zeros(len(atrack))
        atrack['eps_o'] = np.zeros(len(atrack))
        atrack['surf_corr_at_numax'] = np.zeros(len(atrack))
        # #
        for i in range(len(atrack)):
            # historyIndex = atrack[i]['index']
            profileIndex = atrack[i]['profile_number']

            numax_scaling = atrack[i]['numax_scaling']
            g = atrack[i]['star_mass']/atrack[i]['radius']**2.0
            Teff = atrack[i]['Teff']
            feh = atrack[i]['FeH']
            
            atrack['mode_l'][i] = f['profile{:0.0f}/{:s}'.format(profileIndex, 'l')][:][0]
            atrack['mode_n'][i] = f['profile{:0.0f}/{:s}'.format(profileIndex, 'n_pg')][:][0]
            mode_freq = f['profile{:0.0f}/{:s}'.format(profileIndex, 'freq')][:][0]
            mode_inertia = f['profile{:0.0f}/{:s}'.format(profileIndex, 'E_norm')][:][0]
            
            atrack['Dnu_freq_o'][i], atrack['eps_o'][i] = get_model_Dnu(mode_freq, atrack['mode_l'][i], \
                                                    atrack['delta_nu_scaling'][i], atrack['numax_scaling'][i], atrack['mode_n'][i])
            
            thetas = np.array([-3.7443726, 1.0866573, -8.742777 , -1.3795667, 
                               -4.9891973, 1.0541984, -8.15369 , -1.3154854])
                
            D1 = thetas[0] * (g)**thetas[1] * (Teff/5777.)**thetas[2] * (thetas[3]*feh + 1) # surf corr at numax
            D2 = thetas[4] * (g)**thetas[5] * (Teff/5777.)**thetas[6] * (thetas[7]*feh + 1) # surf corr at scale*numax
            scale = 1.1

            fi = scipy.interpolate.interp1d(mode_freq, (mode_freq/numax_scaling)**3.0/mode_inertia, kind='cubic', fill_value='extrapolate')
            gs = fi(numax_scaling)

            fi = scipy.interpolate.interp1d(mode_freq, (mode_freq/numax_scaling)**-1.0/mode_inertia, kind='cubic', fill_value='extrapolate')
            fs = fi(numax_scaling)

            surface_a3 = (scale**-1 * D1 - D2)/ ((scale**-1 - scale**3) *gs)
            surface_a1 = (scale**3 * D1 - D2)/ ((scale**3 - scale**-1) *fs)
            dfreq = (surface_a3 *(mode_freq/numax_scaling)**3.0+surface_a1 *(mode_freq/numax_scaling)**-1.0)/mode_inertia
            
            atrack['mode_freq_o'][i] = mode_freq
            atrack['mode_freq'][i] = mode_freq + dfreq
            atrack['mode_inertia'][i] = mode_inertia
            atrack['surf_corr_at_numax'][i] = D1
            
        return atrack
    

    tracks = process_map(multi, grids.loc[sl,:].iterrows(), total=end-start, max_workers=14)
    data = astropy.table.vstack([track for track in tracks if track is not None])
    np.save('models/calibrated/tracks_{:0.0f}_{:0.0f}'.format(sl.start, sl.stop), data)


slice(0, 999, 1)


  0%|          | 0/999 [00:00<?, ?it/s]

slice(1000, 1999, 1)


  0%|          | 0/999 [00:00<?, ?it/s]

slice(2000, 2999, 1)


...

  0%|          | 0/999 [00:00<?, ?it/s]

slice(7000, 7999, 1)


  0%|          | 0/999 [00:00<?, ?it/s]

slice(8000, 8192, 1)


  0%|          | 0/192 [00:00<?, ?it/s]

## Inverse-cubic, sep sample solution.

In [7]:
from tqdm import tqdm
import astropy.table
from grid import get_model_Dnu
import scipy.interpolate

for i in range(0, 9): #

    start = i*1000
    end = (i+1)*1000-1 if (i+1)*1000-1 < (2**13) else 2**13
    sl = slice(start, end, 1)
    print(sl)
    
    
    def multi(inputs):
        it, track = inputs
        filepath = '../hpc/coarse_v2/pre_rgb_tip/complete_grid/index{:06.0f}.history.h5'.format(track['index'])
        if not os.path.exists(filepath): 
            return None

        cols = ['index', 'profile_number', 'flag_seismo', 
                'Yinit', 'Xinit', 'Zinit', 'fov_core', 'fov_shell', 'amlt', 
                'star_age', 'star_mass', 'luminosity', 'radius', 'Teff', 
                'log_g', 'FeH', 'delta_nu_scaling', 'numax_scaling', 'acoustic_cutoff']

        f = h5py.File(filepath, 'r')
        # print(filepath)
        cols = ['index', 'profile_number', 'flag_seismo', 
                'star_age', 'star_mass', 'luminosity', 'radius', 'Teff', 'delta_nu_scaling', 'numax_scaling', 'acoustic_cutoff', 'delta_Pg', 'log_g',
                'Yinit', 'Xinit', 'Zinit', 'center_h1', 'fov_core', 'fov_shell', 'amlt', 'FeH', 'log_LHe', 'log_Lnuc']


        atrack = Table([f[col][:] for col in cols], names=cols)
        idx = (atrack['flag_seismo']==1) & (atrack['numax_scaling']>15) & ((atrack['log_LHe']-atrack['log_Lnuc'])<-3)

        atrack = atrack[idx]
        atrack['density'] = atrack['star_mass']**0.5/atrack['radius']**-1.5
        atrack['numax'] = atrack['numax_scaling']
        atrack['Dnu'] = atrack['delta_nu_scaling']

        atrack['[M/H]'] = atrack['FeH']
        atrack['mode_l'] = np.zeros(len(atrack), dtype=object)
        atrack['mode_n'] = np.zeros(len(atrack), dtype=object)
        atrack['mode_freq_o'] = np.zeros(len(atrack), dtype=object)
        atrack['mode_freq'] = np.zeros(len(atrack), dtype=object)
        atrack['mode_inertia'] = np.zeros(len(atrack), dtype=object)
        
        # # # replace the acoustic_cutoff from mesa with numax_scaling
        atrack['acoustic_cutoff'] = atrack['numax_scaling']
        atrack['Dnu_freq_o'] = np.zeros(len(atrack))
        atrack['eps_o'] = np.zeros(len(atrack))
        atrack['surf_corr_at_numax'] = np.zeros(len(atrack))
        # #
        for i in range(len(atrack)):
            # historyIndex = atrack[i]['index']
            profileIndex = atrack[i]['profile_number']

            numax_scaling = atrack[i]['numax_scaling']
            g = atrack[i]['star_mass']/atrack[i]['radius']**2.0
            Teff = atrack[i]['Teff']
            feh = atrack[i]['FeH']
            
            atrack['mode_l'][i] = f['profile{:0.0f}/{:s}'.format(profileIndex, 'l')][:][0]
            atrack['mode_n'][i] = f['profile{:0.0f}/{:s}'.format(profileIndex, 'n_pg')][:][0]
            mode_freq = f['profile{:0.0f}/{:s}'.format(profileIndex, 'freq')][:][0]
            mode_inertia = f['profile{:0.0f}/{:s}'.format(profileIndex, 'E_norm')][:][0]
            
            atrack['Dnu_freq_o'][i], atrack['eps_o'][i] = get_model_Dnu(mode_freq, atrack['mode_l'][i], \
                                                    atrack['delta_nu_scaling'][i], atrack['numax_scaling'][i], atrack['mode_n'][i])
            
            ifrgb = (Teff<5500) & (numax_scaling < 260)
            if ifrgb:
                thetas = np.array([-3.8369734, 1.0976515, -8.83042 , -1.3841832, 
                                   -5.0435867, 1.0658274, -8.325036 , -1.3250351])
            else:
                thetas = np.array([-3.715097 , 0.6096728 , -1.6813153 , -0.08168457, 
                                   -5.550722 , 0.6479634 , -1.5653569 , -0.64631474])
            
            # thetas = np.array([-4.124002 , 0.9858308 , -7.428942 , 0.09117827, 
            #                    -5.4557753 , 0.9639112 , -6.7885838 , -0.15924262])
                
            D1 = thetas[0] * (g)**thetas[1] * (Teff/5777.)**thetas[2] * (thetas[3]*feh + 1) # surf corr at numax
            D2 = thetas[4] * (g)**thetas[5] * (Teff/5777.)**thetas[6] * (thetas[7]*feh + 1) # surf corr at scale*numax
            scale = 1.1

            fi = scipy.interpolate.interp1d(mode_freq, (mode_freq/numax_scaling)**3.0/mode_inertia, kind='cubic', fill_value='extrapolate')
            gs = fi(numax_scaling)

            fi = scipy.interpolate.interp1d(mode_freq, (mode_freq/numax_scaling)**-1.0/mode_inertia, kind='cubic', fill_value='extrapolate')
            fs = fi(numax_scaling)

            surface_a3 = (scale**-1 * D1 - D2)/ ((scale**-1 - scale**3) *gs)
            surface_a1 = (scale**3 * D1 - D2)/ ((scale**3 - scale**-1) *fs)
            dfreq = (surface_a3 *(mode_freq/numax_scaling)**3.0+surface_a1 *(mode_freq/numax_scaling)**-1.0)/mode_inertia
            
            atrack['mode_freq_o'][i] = mode_freq
            atrack['mode_freq'][i] = mode_freq + dfreq
            atrack['mode_inertia'][i] = mode_inertia
            atrack['surf_corr_at_numax'][i] = D1
            
        return atrack
    

    tracks = process_map(multi, grids.loc[sl,:].iterrows(), total=end-start, max_workers=14)
    data = astropy.table.vstack([track for track in tracks if track is not None])
    np.save('models/calibratedsep/tracks_{:0.0f}_{:0.0f}'.format(sl.start, sl.stop), data)


slice(0, 999, 1)


  0%|          | 0/999 [00:00<?, ?it/s]

slice(1000, 1999, 1)


  0%|          | 0/999 [00:00<?, ?it/s]

slice(2000, 2999, 1)


...

  0%|          | 0/999 [00:00<?, ?it/s]

slice(6000, 6999, 1)


  0%|          | 0/999 [00:00<?, ?it/s]

slice(8000, 8192, 1)


  0%|          | 0/192 [00:00<?, ?it/s]